# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

The study's objective was to investigate the molecular/transcriptomic effects of oxycodon (or in generell opioid) withdrawal from mice suffering from peripheral nerve injury compared to control mice absent of chronic neuropathic pain in the  brain reward circuitry.

What do the conditions mean?

oxy: Treatment with oxycodon.


sal: Treatment with saline, acting as control.

What do the genotypes mean?

SNI: Spared Nerve Injury Genotype (nerval removal) -> Chronic pair


Sham: Control without special genotype (surgery without nerve cutting was performed, so e.g. skin pain is comparable) -> Surgery effects, but no chronic pain.

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?

- Basic RNA Seq analysis workflow:
- Quality control, adapter trimming, alignment against reference, post-analysis quality control etc.
- Either do it manually, or look for pipelines for example in nf-core

Which groups would you compare to each other?

- SNI-oxy vs sham-sal
- SNI-sal vs sham-sal
- sham-oxy vs sham-sal
- Compare all non-control genotypes + treatments to control (sham-sal) to eventually identify effects that are really based on the oxycodon withdrawal during chronic pain by ruling out effects that may arise from oxycodon treatment and withdrawal without condition itself or the SNI genotype.

Please also mention which outcome you would expect to see from each comparison. 

- SNI-oxy vs sham-sal: Withdrawal in presenence of chronic pain 
- SNI-sal vs sham-sal: Disease only
- Sham-oxy vs sham-sal: Withdrawal only

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [50]:
import pandas as pd
import numpy as np

# Import and sort fancy table
df = pd.read_csv("/Users/nilswaffenschmidt/waffenschmidt/notebooks/day_02/conditions_runs_oxy_project.csv", sep=";")
df = df.fillna(False).replace("x", True)

# Wewrite conditions
coniditons = ["Sal", "Oxy"]
genotypes  = ["SNI", "Sham"]
df["Condition"] = np.select(df[["condition: Sal", "Condition: Oxy"]].to_numpy().T, coniditons, default=None)
df["Genotype"]  = np.select(df[["Genotype: SNI", "Genotype: Sham"]].to_numpy().T, genotypes, default=None)
df = df.drop(['condition: Sal', 'Condition: Oxy', 'Genotype: SNI', 'Genotype: Sham'], axis=1)

# Sort by Conditions and Genotypes
df = df.sort_values(by=['Condition', 'Genotype'])
df

/var/folders/8f/g9m13fjx2rjd968dsn9tzvcm0000gn/T/ipykernel_39961/72308387.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False).replace("x", True)
/var/folders/8f/g9m13fjx2rjd968dsn9tzvcm0000gn/T/ipykernel_39961/72308387.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False).replace("x", True)


,Patient,Run,RNA-seq,DNA-seq,Condition,Genotype
3,?,SRR23195508,True,False,Oxy,SNI
4,?,SRR23195509,True,False,Oxy,SNI
11,?,SRR23195516,True,False,Oxy,SNI
12,?,SRR23195517,True,False,Oxy,SNI
1,?,SRR23195506,True,False,Oxy,Sham
6,?,SRR23195511,True,False,Oxy,Sham
9,?,SRR23195514,True,False,Oxy,Sham
14,?,SRR23195519,True,False,Oxy,Sham
0,?,SRR23195505,True,False,Sal,SNI
5,?,SRR23195510,True,False,Sal,SNI


We have 8 samples per condition. 8 samples per genotype. 4 samples per condition per genotype.

They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [48]:
# Add Bases to fancy table
df_counts = pd.read_csv("/Users/nilswaffenschmidt/waffenschmidt/notebooks/day_02/base_counts.csv", sep=",")
df_final  = pd.merge(left=df, right=df_counts, on="Run")
df_final  = df_final.sort_values(by="Bases", ascending=True)
df_final  = df_final.sort_values(['Bases'],ascending=False)

df_final

,Patient,Run,RNA-seq,DNA-seq,Condition,Genotype,Bases
14,?,SRR23195515,True,False,Sal,Sham,8169101700
10,?,SRR23195513,True,False,Sal,SNI,8099181600
12,?,SRR23195507,True,False,Sal,Sham,8063298900
11,?,SRR23195518,True,False,Sal,SNI,7908500400
4,?,SRR23195506,True,False,Oxy,Sham,7859530800
15,?,SRR23195520,True,False,Sal,Sham,7858146000
13,?,SRR23195512,True,False,Sal,Sham,7462857900
9,?,SRR23195510,True,False,Sal,SNI,7377388500
6,?,SRR23195514,True,False,Oxy,Sham,7226808600
1,?,SRR23195509,True,False,Oxy,SNI,7003550100


In [49]:
!nextflow run nf-core/fetchngs --input /Users/nilswaffenschmidt/waffenschmidt/notebooks/day_02/ids.csv -profile docker,arm --max_memory "4GB" --outdir /Users/nilswaffenschmidt/waffenschmidt/notebooks/day_02/nf_output


 N E X T F L O W   ~  version 25.04.7

Pulling nf-core/fetchngs ...
WARN: Cannot read project manifest -- Cause: API rate limit exceeded -- Provide your GitHub user name and password to get a higher rate limit
API rate limit exceeded -- Provide your GitHub user name and password to get a higher rate limit


While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.